## Bribes research

In [34]:
import math

import requests

from notebooks.bribes_research.helpers import fetch_token_prices

BADGER_BRIBE = 40000

BRIBES_URL = "https://api.llama.airforce/bribes"
DASHBOARD_URL = "https://api.llama.airforce//dashboard"

token_prices = fetch_token_prices(['badger-dao'])
badger_price = token_prices['badger-dao']['usd']
badger_bribe_in_dollar = BADGER_BRIBE * badger_price

cvx_bribes = requests.post(BRIBES_URL, json={'platform': "votium", 'protocol': "cvx-crv"}).json()
cvx_dash = requests.post(DASHBOARD_URL, json={"id": "bribes-overview-votium"}).json()
aura_bribes = requests.post(BRIBES_URL, json={'platform': "hh", 'protocol': "aura-bal"}).json()
aura_dash = requests.post(DASHBOARD_URL, json={"id": "bribes-overview-aura"}).json()

total_bribes_cvx = sum([bribe['amountDollars'] for bribe in cvx_bribes['epoch']['bribes']])
dollar_spent_on_bribes_cvx = cvx_dash['dashboard']['rewardPerDollarBribe']

total_bribes_aura = sum([bribe['amountDollars'] for bribe in aura_bribes['epoch']['bribes']])
dollar_spent_on_bribes_aura = aura_dash['dashboard']['rewardPerDollarBribe']

cvx_dilution_100 = badger_bribe_in_dollar / total_bribes_cvx * 100
aura_dilution_100 = badger_bribe_in_dollar / total_bribes_aura * 100

cvx_in_out_100 = (100 - cvx_dilution_100) * dollar_spent_on_bribes_cvx
aura_in_out_100 = (100 - aura_dilution_100) * dollar_spent_on_bribes_aura

print(f"CVX $I/O if consuming 100% bribes is {cvx_in_out_100}%")
print(f"AURA $I/O if consuming 100% bribes is {aura_in_out_100}%")

# Rebalancing bribe values
cvx_dilution_50 = ((badger_bribe_in_dollar / 2) / total_bribes_cvx * 100)
aura_dilution_50 = ((badger_bribe_in_dollar / 2) / total_bribes_aura * 100)

cvx_in_out_50 = (100 - cvx_dilution_50) * dollar_spent_on_bribes_cvx
aura_in_out_50 = (100 - aura_dilution_50) * dollar_spent_on_bribes_aura
print(f"CVX $I/O if consuming 50% bribes is {cvx_in_out_50}%")
print(f"AURA $I/O if consuming 50% bribes is {aura_in_out_50}%")

# Rebalance here
cvx_in_out_floating = cvx_in_out_50
aura_in_out_floating = aura_in_out_50
rebalancing_step = 0.01  # in %
cvx_bribe_portion = aura_bribe_portion = badger_bribe_in_dollar / 2
while not math.isclose(cvx_in_out_floating, aura_in_out_floating, abs_tol=0.5):
    if cvx_in_out_floating > aura_in_out_floating:
        # Take bribe portion from aura and give it to cvx
        bribe_stolen = aura_bribe_portion * rebalancing_step
        aura_bribe_portion -= bribe_stolen
        cvx_bribe_portion += bribe_stolen
    elif aura_in_out_floating > cvx_in_out_floating:
        # Take bribe portion from cvx and give it to aura
        bribe_stolen = aura_bribe_portion * rebalancing_step
        cvx_bribe_portion -= bribe_stolen
        aura_bribe_portion += bribe_stolen
    cvx_dilution =  cvx_bribe_portion / total_bribes_cvx * 100
    aura_dilution = aura_bribe_portion / total_bribes_aura * 100
    cvx_in_out_floating = (100 - cvx_dilution) * dollar_spent_on_bribes_cvx
    aura_in_out_floating = (100 - aura_dilution) * dollar_spent_on_bribes_aura

print(f"Estimated BADGER bribe for CVX: {cvx_bribe_portion}$. Percent from total bribes: {round(cvx_bribe_portion / badger_bribe_in_dollar * 100, 2)}%. $I/O: {cvx_in_out_floating}")
print(f"Estimated BADGER bribe for AURA: {aura_bribe_portion}$. Percent from total bribes: {round(aura_bribe_portion / badger_bribe_in_dollar * 100, 2)}%. $I/O: {aura_in_out_floating}")

CVX $I/O if consuming 100% bribes is 137.83588649666675%
AURA $I/O if consuming 100% bribes is 111.92339066855205%
CVX $I/O if consuming 50% bribes is 142.50119346737745%
AURA $I/O if consuming 50% bribes is 180.29249415360653%
Estimated BADGER bribe for CVX: 40902.85895616573$. Percent from total bribes: 24.06%. $I/O: 144.9215075791447
Estimated BADGER bribe for AURA: 129097.14104383427$. Percent from total bribes: 75.94%. $I/O: 144.82329416041887
